## Problem 1

In [365]:
import pickle
import pandas as pd

# 加载 pickle 文件中的数据
with open('./dataset/probability_matrices.pkl', 'rb') as f:
    A_data = pickle.load(f)

with open('./dataset/number_matrices.pkl', 'rb') as f:
    Pn_data = pickle.load(f)

# 加载东道主变量, csv 文件中的数据
host_data = pd.read_csv('./dataset/host_or_not.csv', index_col=0)

# 填充 A 和 Pn 数据中的缺失值为 0
A_data = {year: df.fillna(0) for year, df in A_data.items()}
Pn_data = {year: df.fillna(0) for year, df in Pn_data.items()}

stable_countries = [
    "AUS", "BRA", "CAN", "DEN", "FIN", "FRA", "GER", "GBR", 
    "HUN", "IRI", "ITA", "JPN", "NED", "NZL", "KOR", "ESP", 
    "USA", "CHN", "ARG", "AUT", "BEL", "BUL", "CUB", 
    "GRE", "IND", "JAM", "MEX", "NOR", 
    "POL", "POR", "ROU", "RSA", "SWE", 
    "SUI", "TUR", "URS", "RUS"
]

# 选择稳定国家的数据
A_data = {year: df.loc[stable_countries] for year, df in A_data.items()}
Pn_data = {year: df.loc[stable_countries] for year, df in Pn_data.items()}



C:\Users\Maqin\AppData\Local\Temp\ipykernel_18068\1653068871.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  A_data = {year: df.fillna(0) for year, df in A_data.items()}
C:\Users\Maqin\AppData\Local\Temp\ipykernel_18068\1653068871.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Pn_data = {year: df.fillna(0) for year, df in Pn_data.items()}


In [351]:
# 打印年份
print(A_data.keys())
print(Pn_data.keys())

dict_keys([1896, 1900, 1904, 1906, 1908, 1912, 1920, 1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020, 2024])
dict_keys([1896, 1900, 1904, 1906, 1908, 1912, 1920, 1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020, 2024])


In [353]:
import numpy as np
import pandas as pd

# 假设 A_data 和 Pn_data 仍然是字典，键是年份，值是 DataFrame
# 获取年份的列表（假设数据中年份是离散的）
years = sorted(A_data.keys())  # 按索引排序年份

# 计算特征矩阵 X 和目标矩阵 y
X = []  # 存储特征
y = []  # 存储目标

# 按年份索引处理数据
for i in range(3, len(years)):  # 从第二年开始处理
    year_prev = years[i - 1]  # 前一届
    year_curr = years[i]  # 当前届
    
    # 获取前一届和当前届的 A 和 Pn 数据
    A_prev_1 = A_data[year_prev]
    A_curr = A_data[year_curr]
    Pn_prev_1 = Pn_data[year_prev]
    Pn_curr = Pn_data[year_curr]
    
    # 计算参赛人数差（Pn - Pn'）
    Pn_diff_1 = Pn_curr - Pn_prev_1
    Pn_diff_1 = Pn_diff_1.values.flatten()

    '''#将Pn_diff标准化
    Pn_diff_1 = (Pn_diff_1 - np.mean(Pn_diff_1)) / np.std(Pn_diff)'''
    
    # 获取前三届获奖比例的平均值
    A_prev_3 = (A_data[years[i - 3]] + A_data[years[i - 2]] + A_data[years[i - 1]]) / 3
    A_prev_3 = A_prev_3.values.flatten()
    
    # 获取当前参赛人数和前三届平均参赛人数的差
    Pn_diff_3 = Pn_curr - (Pn_data[years[i - 3]] + Pn_data[years[i - 2]] + Pn_data[years[i - 1]]) / 3
    Pn_diff_3 = Pn_diff_3.values.flatten()

    # 将特征（上一年的获奖比例和参赛人数差）展平并加入 X
    X.append(np.concatenate([A_prev_1.values.flatten(), Pn_diff_1, A_prev_3, Pn_diff_3, host_data[str(year_curr)].values.repeat(55)]))
    
    # 将当前年获奖比例展平并加入 y
    y.append(A_curr.values.flatten())

# 转换为 numpy 数组，便于后续处理
X = np.array(X)
y = np.array(y)


In [354]:
from sklearn.model_selection import train_test_split

# 按照时间顺序划分训练集和测试集
train_idx, test_idx = train_test_split(np.arange(len(years)-3), test_size=0.1, shuffle=True)

# 根据索引划分训练集和测试集
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

In [355]:
print(train_idx)
print(test_idx)

[12 22 10 23  0 24  4 14 20 25 18 27 19  8 11  9  7  6 26  1 13 21  3  5
  2]
[15 16 17]


In [356]:
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# 构建 DMatrix（XGBoost 使用的数据格式）
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.01)
# xgb_model = xgb.XGBRegressor(objective='reg:absoluteerror', n_estimators=100, max_depth=3, learning_rate=0.01)
multi_output_model = MultiOutputRegressor(xgb_model)

# 训练模型
multi_output_model.fit(X_train, y_train)

# 预测
y_pred = multi_output_model.predict(X_test)

# 评估模型，使用mae
mae = np.mean(np.abs(y_test - y_pred))
print(f'MAE: {mae:.4f}')

# 评估模型，使用mse
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse:.4f}')


MAE: 0.0129
MSE: 0.0039


In [357]:
N, M = A_curr.shape
print(f"Number of rows: {N}, number of columns: {M}")

Number of rows: 37, number of columns: 55


In [358]:
y_pred = multi_output_model.predict(X)

In [359]:
print(y_pred.shape)

(28, 2035)


In [360]:
import dataprocess
programms_num = dataprocess.get_programms_num()
programms_num = programms_num.iloc[:, 3:].to_numpy()
programms_num = np.nan_to_num(programms_num)
print(programms_num.shape)

(55, 28)


In [361]:
import pandas as pd
# 通过向量内积保存每一年各个国家预测金牌数和实际金牌数
for i in range(len(years) - 3):
    predict_gold_medals = np.dot(y_pred[i].reshape(N, M) , programms_num[:, i])
    actual_gold_medals = np.dot(y[i].reshape(N, M) , programms_num[:, i])
    # print(f"Predicted gold medals: {predict_gold_medals}")
    # print(f"Actual gold medals: {actual_gold_medals}")
    # 转换为DataFrame后前面加上国家名
    predict_gold_medals = pd.DataFrame(predict_gold_medals, index=stable_countries, columns=['Predicted gold medals'])
    actual_gold_medals = pd.DataFrame(actual_gold_medals, index=stable_countries, columns=['Actual gold medals'])
    # 合并为一个DataFrame
    result = pd.concat([predict_gold_medals, actual_gold_medals], axis=1)
    # 保存为csv文件
    result.to_csv(f'./output/{years[i+3]}.csv')
    

In [362]:
# 计算每年的预测金牌数和实际金牌数的平均绝对误差
mae = 0

for i in range(len(years) - 3):
    predict_gold_medals = np.dot(y_pred[i].reshape(N, M) , programms_num[:, i])
    actual_gold_medals = np.dot(y[i].reshape(N, M) , programms_num[:, i])
    mae += np.abs(predict_gold_medals - actual_gold_medals)

mae /= len(years) - 3
print(mae)

# 累加mae
mae = mae.sum()
print(mae)

[1.99686098 0.62000541 1.16760358 0.59628648 1.74925253 1.70629626
 3.20960899 3.81599768 1.56271084 0.41746456 1.59341469 2.02291369
 1.34861782 0.89298724 1.59118664 0.8183556  6.06908256 4.75264066
 0.5912174  0.47906445 0.75303366 1.09666261 1.61131802 0.63821098
 0.23842506 0.50961005 0.3575473  0.65547168 1.03240883 0.14564578
 1.57976386 0.55363335 2.60435262 0.77909    0.79171757 9.58407479
 3.27894253]
63.211476769792036


In [367]:
A_prev_1 = A_data[years[-1]]
# 假设Pn_diff_1为0
Pn_diff_1 = np.zeros((N, M))


# 获取前三届获奖比例的平均值
A_prev_3 = (A_data[years[-3]] + A_data[years[-2]] + A_data[years[-1]]) / 3
A_prev_3 = A_prev_3.values.flatten()

# 获取当前参赛人数和前三届平均参赛人数的差
Pn_diff_3 = Pn_data[years[-1]] - (Pn_data[years[-3]] + Pn_data[years[-2]] + Pn_data[years[-1]]) / 3
Pn_diff_3 = Pn_diff_3.values.flatten()

In [389]:
X_2028 = np.concatenate([A_prev_1.values.flatten(), Pn_diff_1.flatten(), A_prev_3, Pn_diff_3, host_data[str(2028)].values.repeat(55)])

In [390]:
# 预测2028年的金牌数
y_2028 = multi_output_model.predict([X_2028])
y_2028 = y_2028.reshape(N, M)

programms_num_2028 = dataprocess.get_programms_num().iloc[:, -1].to_numpy()
programms_num_2028 = np.nan_to_num(programms_num_2028)

predict_gold_medals_2028 = np.dot(y_2028, programms_num_2028)
print(predict_gold_medals_2028)


[ 7.11477427  1.27725502  1.61885967  2.06566824  2.36208705  9.29302265
  8.86783601 10.52995394  6.38128311  0.3365656   8.6567872   9.70870752
  4.61418534  2.44175125  4.36381181  1.01021025 44.482565   22.38022032
  0.76208463  0.60444325  1.52826564  1.99341381  1.143099    0.59739402
  0.24670955  3.17419982  0.13527607  2.37294287  2.37836276  0.4831005
  2.81071224  0.55609922  5.66204702  1.08207111  0.66617952  5.41978842
  4.10668499]


In [388]:
# 保存predict_gold_medals_2028
predict_gold_medals_2028 = pd.DataFrame(predict_gold_medals_2028, index=stable_countries, columns=['Predicted gold medals'])
predict_gold_medals_2028.to_csv(f'./output/2028_predict_gold_medals.csv')

In [391]:
# y_2028中，若对应programms_num_2028的值为0，则将预测金牌比例置为0
for i in range(len(programms_num_2028)):
    if programms_num_2028[i] == 0:
        y_2028[:, i] = 0

In [393]:
predict_gold_medals_2028 = np.dot(y_2028, programms_num_2028)
print(predict_gold_medals_2028)

[ 7.11477427  1.27725502  1.61885967  2.06566824  2.36208705  9.29302265
  8.86783601 10.52995394  6.38128311  0.3365656   8.6567872   9.70870752
  4.61418534  2.44175125  4.36381181  1.01021025 44.482565   22.38022032]


In [392]:
# 只要前17行
y_2028 = y_2028[:18]

# 将y_2028转换为dataframe保存入csv文件
y_2028 = pd.DataFrame(y_2028)
y_2028.to_csv(f'./output/2028_change.csv')
